In [ ]:
# Bringing in tensorflow
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
import argparse
import os
import numpy as np
import math
import sys
import matplotlib.pyplot as plt
import helper
import torchvision
import torchvision.transforms as T
from torchvision.utils import save_image
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
from tqdm.notebook import tqdm
import torch.nn.functional as F
import torch.nn as nn
import torch.nn.functional as F
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
project_path = "/content/drive/MyDrive/GAN/cartoon"
os.chdir(project_path)

Mounted at /content/drive


In [ ]:
train_data_dir = os.path.join(project_path, "train")
test_data_dir = os.path.join(project_path, "test")


In [ ]:
image_size = 64
batch_size = 64
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

In [ ]:
trainloader = datasets.ImageFolder(train_data_dir, transform=T.Compose([
    T.Resize(image_size),
    T.CenterCrop(image_size),
    T.ToTensor(),
    T.Normalize(*stats)]))

trainloader = DataLoader(trainloader, batch_size, shuffle=True, num_workers=3, pin_memory=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
testloader = datasets.ImageFolder(test_data_dir, transform=T.Compose([
    T.Resize(image_size),
    T.CenterCrop(image_size),
    T.ToTensor(),
    T.Normalize(*stats)]))

testloader = DataLoader(testloader, batch_size, shuffle=True, num_workers=3, pin_memory=True)

In [ ]:
# # Задание параметров
# image_size = 64
# batch_size = 64
# stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

# # Создание загрузчика данных
# trainloader = datasets.ImageFolder(
#     train_data_dir,
#     transform=T.Compose([
#         T.Resize(image_size),
#         T.CenterCrop(image_size),
#         T.ToTensor(),
#         T.Normalize(*stats)
#     ])
# )

# trainloader = DataLoader(trainloader, batch_size=batch_size, shuffle=True, num_workers=3, pin_memory=True)

# # Создание модели TensorFlow
# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
#     tf.keras.layers.MaxPooling2D((2, 2)),
#     tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D((2, 2)),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(10)
# ])

# # Извлечение изображений из DataLoader и передача в модель
# dataiter = iter(trainloader)
# images, labels = next(dataiter)

# # Конвертация изображений в numpy массив
# images_np = images.numpy()

# # Транспонирование изображений, чтобы соответствовать TensorFlow
# images_np = np.transpose(images_np, (0, 2, 3, 1))

# # Передача изображений в модель
# output = model(images_np)
# print(output)

In [ ]:
# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

# epochs = 10
# for epoch in range(epochs):
#     for images, labels in trainloader:
#         images_np = images.numpy()
#         images_np = np.transpose(images_np, (0, 2, 3, 1))
#         labels_np = labels.numpy()

#         model.train_on_batch(images_np, labels_np)
#         # Выводим информацию о текущем батче
#         print(f"Epoch {epoch+1}/{epochs}, Batch processed")

In [ ]:
# testloader = datasets.ImageFolder(test_data_dir, transform=T.Compose([
#     T.Resize(image_size),
#     T.CenterCrop(image_size),
#     T.ToTensor(),
#     T.Normalize(*stats)
# ]))

# testloader = DataLoader(testloader, batch_size=batch_size, shuffle=False, num_workers=3, pin_memory=True)

# for images, labels in testloader:
#     images_np = images.numpy()
#     images_np = np.transpose(images_np, (0, 2, 3, 1))
#     labels_np = labels.numpy()

#     test_loss, test_acc = model.evaluate(images_np, labels_np, verbose=2)
#     print(f"Test accuracy: {test_acc}, Test loss: {test_loss}")
#     break  # Удалите этот break для оценки на всем тестовом наборе

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img,title):
    img = img / 2 + 0.5
    npimg = img.numpy()

    plt.figure(figsize=(8,8))
    plt.title(title)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis('off')
    plt.show()

dataiter = iter(trainloader)
for images, labels in dataiter:
    # show images
    imshow(torchvision.utils.make_grid(images), "Training images")
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)
    break

In [ ]:
# Извлечение изображений из dataiter
dataiter = iter(trainloader)
images = []
for i, batch in enumerate(dataiter):
    imgs, _ = batch  # Извлечение изображений, игнорирование меток
    images.append(imgs)
    if i >= 1:  # Обработаем два батча для проверки
        break

# Конкатенируем все изображения в один массив
images = torch.cat(images)

# Преобразование изображений в numpy массив
images_np = images.numpy()

# Проверим размеры numpy массива
print("Images numpy shape:", images_np.shape)

In [ ]:
images_np = images.numpy()
images_np = np.transpose(images_np, (0, 2, 3, 1))  # Преобразуем из [N, C, H, W] в [N, H, W, C]
print("Transposed images numpy shape:", images_np.shape)

images_tf = tf.convert_to_tensor(images_np, dtype=tf.float32)
print("TensorFlow tensor shape:", images_tf.shape)

dataset = tf.data.Dataset.from_tensor_slices(images_tf).batch(64)

Build Neural Network


In [ ]:
# Bring in the sequential api for the generator and discriminator
from tensorflow.keras.models import Sequential
# Bring in the layers for the neural network
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Reshape, LeakyReLU, Dropout, UpSampling2D

In [ ]:
def build_generator():
    model = Sequential()
    model.add(Dense(8 * 8 * 256, input_dim=256))
    model.add(LeakyReLU(0.2))
    model.add(Reshape((8, 8, 256)))

    # Upsampling block 1
    model.add(UpSampling2D())
    model.add(Conv2D(128, 5, padding='same'))
    model.add(LeakyReLU(0.2))

    # Upsampling block 2
    model.add(UpSampling2D())
    model.add(Conv2D(64, 5, padding='same'))
    model.add(LeakyReLU(0.2))

    # Additional upsampling block to reach (64, 64)
    model.add(UpSampling2D())
    model.add(Conv2D(32, 5, padding='same'))
    model.add(LeakyReLU(0.2))

    # Convolutional block 1
    model.add(Conv2D(32, 4, padding='same'))
    model.add(LeakyReLU(0.2))

    # Convolutional block 2
    model.add(Conv2D(16, 4, padding='same'))
    model.add(LeakyReLU(0.2))

    # Conv layer to get to three channels (RGB)
    model.add(Conv2D(3, 3, padding='same', activation='sigmoid'))
    return model

In [ ]:
# img = generator.predict(np.random.randn(4,256,1))
random_data = np.random.randn(4, 256)
generator = build_generator()
generated_images = generator.predict(random_data)
print("Generated Images shape:", generated_images.shape)

1/1 [==============================] - 2s 2s/step
Generated Images shape: (4, 64, 64, 3)


In [ ]:
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16384)             4210688   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 16384)             0         
                                                                 
 reshape (Reshape)           (None, 8, 8, 256)         0         
                                                                 
 up_sampling2d (UpSampling2  (None, 16, 16, 256)       0         
 D)                                                              
                                                                 
 conv2d (Conv2D)             (None, 16, 16, 128)       819328    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                        

In [ ]:
# Generate
img = generator.predict(np.random.randn(4,256))
# Setup the subplot formatting
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
# Loop four times and get images
for idx, img in enumerate(img):
    # Plot the image using a specific subplot
    ax[idx].imshow(np.squeeze(img))
    # Appending the image label as the plot title
    ax[idx].title.set_text(idx)

Discriminator епта

In [ ]:
def build_discriminator():
    model = Sequential()

    # First Conv Block
    model.add(Conv2D(32, (5, 5), input_shape=(64, 64, 3), padding='same'))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))

    # Second Conv Block
    model.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))

    # Third Conv Block
    model.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))

    # Fourth Conv Block
    model.add(Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))

    # Flatten then pass to dense layer
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [ ]:
generated_images_resized = np.zeros((4, 64, 64, 3))
for i in range(4):
    generated_images_resized[i] = tf.image.resize(generated_images[i], (64, 64)).numpy()

# Создание дискриминатора
discriminator = build_discriminator()

# Подготовка данных для дискриминатора
discriminator_input = np.repeat(generated_images_resized, 64, axis=0)  # Повторяем изображения 64 раза для создания батча
print("Discriminator Input shape:", discriminator_input.shape)

# Подача изображения в дискриминатор
discriminator_output = discriminator.predict(discriminator_input)
print("Discriminator Output shape:", discriminator_output.shape)

Discriminator Input shape: (256, 64, 64, 3)
8/8 [==============================] - 0s 3ms/step
Discriminator Output shape: (256, 1)


In [ ]:
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 64, 64, 32)        2432      
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 64, 64, 32)        0         
                                                                 
 dropout (Dropout)           (None, 64, 64, 32)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 32, 32, 64)        51264     
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 32, 32, 64)        0         
                                                                 
 dropout_1 (Dropout)         (None, 32, 32, 64)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 16, 16, 128)      

In [ ]:
# discriminator_output

In [ ]:
# img = np.repeat(img[:, :, np.newaxis], 3, axis=2)  # Копируем канал три раза для RGB
# img = np.expand_dims(img, axis=0)  # Добавляем размерность батча (батч, высота, ширина, канал)

Construct Training Loop.
Setup Losses and Optimizers

In [ ]:
# Adam is going to be the optimizer for both
from tensorflow.keras.optimizers import Adam
# Binary cross entropy is going to be the loss for both
from tensorflow.keras.losses import BinaryCrossentropy

g_opt = Adam(learning_rate=0.0001)
d_opt = Adam(learning_rate=0.00001)
g_loss = BinaryCrossentropy()
d_loss = BinaryCrossentropy()

Build Subclassed Model

In [ ]:
# Importing the base model class to subclass our training step
from tensorflow.keras.models import Model

In [ ]:
class PumbaGAN(Model):
    def __init__(self, generator, discriminator, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.generator = generator
        self.discriminator = discriminator

    def compile(self, g_opt, d_opt, g_loss, d_loss, *args, **kwargs):
        super().compile(*args, **kwargs)
        self.g_opt = g_opt
        self.d_opt = d_opt
        self.g_loss = g_loss
        self.d_loss = d_loss

    def train_step(self, batch):
        real_images = batch[0]
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, 256))

        # Generate fake images
        fake_images = self.generator(random_latent_vectors, training=True)

        # Train the discriminator
        with tf.GradientTape() as d_tape:
            yhat_real = self.discriminator(real_images, training=True)
            yhat_fake = self.discriminator(fake_images, training=True)
            yhat_realfake = tf.concat([yhat_real, yhat_fake], axis=0)
            y_realfake = tf.concat([tf.zeros_like(yhat_real), tf.ones_like(yhat_fake)], axis=0)
            noise_real = 0.15 * tf.random.uniform(tf.shape(yhat_real))
            noise_fake = -0.15 * tf.random.uniform(tf.shape(yhat_fake))
            y_realfake += tf.concat([noise_real, noise_fake], axis=0)
            total_d_loss = self.d_loss(y_realfake, yhat_realfake)

        dgrad = d_tape.gradient(total_d_loss, self.discriminator.trainable_variables)
        self.d_opt.apply_gradients(zip(dgrad, self.discriminator.trainable_variables))

        # Train the generator
        with tf.GradientTape() as g_tape:
            gen_images = self.generator(random_latent_vectors, training=True)
            predicted_labels = self.discriminator(gen_images, training=False)
            total_g_loss = self.g_loss(tf.zeros_like(predicted_labels), predicted_labels)

        ggrad = g_tape.gradient(total_g_loss, self.generator.trainable_variables)
        self.g_opt.apply_gradients(zip(ggrad, self.generator.trainable_variables))

        return {"d_loss": total_d_loss, "g_loss": total_g_loss}


In [ ]:
pumba = PumbaGAN(generator, discriminator)

In [ ]:
pumba.compile(g_opt, d_opt, g_loss, d_loss)

Build Callback

In [ ]:
import os
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import Callback

In [ ]:
class ModelMonitor(Callback):
    def __init__(self, num_img=3, latent_dim=256, img_size=(64, 64, 3)):
        self.num_img = num_img
        self.latent_dim = latent_dim
        self.img_size = img_size

    def on_epoch_end(self, epoch, logs=None):
        # Generate random latent vectors
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))

        # Generate images using the generator
        generated_images = self.model.generator(random_latent_vectors)

        # Scale images from [-1, 1] to [0, 1]
        generated_images = (generated_images + 1) / 2.0
        generated_images = tf.clip_by_value(generated_images, 0, 1)

        # Convert to uint8
        generated_images = (generated_images * 255).numpy().astype('uint8')

        # Save the generated images
        for i in range(self.num_img):
            img = array_to_img(generated_images[i])
            img.save(os.path.join('images', f'generated_img_{epoch}_{i}.png'))


Train

In [ ]:
def torch_to_tf_data(loader):
    def gen():
        for batch in loader:
            images, _ = batch
            images = images.permute(0, 2, 3, 1).numpy()  # Convert to HWC format
            yield images

    tf_data = tf.data.Dataset.from_generator(
        gen,
        output_signature=tf.TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32)
    )

    return tf_data

# Create TensorFlow datasets
tf_train_dataset = torch_to_tf_data(trainloader).batch(batch_size).prefetch(1)
tf_test_dataset = torch_to_tf_data(testloader).batch(batch_size).prefetch(1)

In [ ]:
hist = pumba.fit(tf_train_dataset, epochs=20, callbacks=[ModelMonitor()])

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) UNKNOWN:  OSError: Caught OSError in DataLoader worker process 1.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torchvision/datasets/folder.py", line 245, in __getitem__
    sample = self.loader(pat

In [ ]:
import tensorflow as tf

def preprocess_image(image, label):
    try:
        image = tf.image.resize(image, [image_size, image_size])
        image = (image - 0.5) / 0.5  # Normalize to [-1, 1]
        return image, label
    except Exception as e:
        # Return a blank image and skip this one
        image = tf.zeros([image_size, image_size, 3])
        return image, label

def load_dataset(directory):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        directory,
        labels='inferred',
        label_mode='int',
        image_size=(image_size, image_size),
        batch_size=batch_size
    )
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return dataset

train_dataset = load_dataset(train_data_dir)
test_dataset = load_dataset(test_data_dir)

# Prefetch for performance
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

hist = pumba.fit(train_dataset, epochs=20, callbacks=[ModelMonitor()])


Found 4155 files belonging to 6 classes.
Found 512 files belonging to 6 classes.
Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) INVALID_ARGUMENT:  Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]]
	 [[Shape/_8]]
  (1) INVALID_ARGUMENT:  Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_5996]